In [1]:
import pandas as pd
from sqlalchemy.orm import sessionmaker#,relationship, backref
from sqlalchemy import create_engine,select
#from Schema import *
from utils_modeling import *

In [2]:
def get_metric_ts(metrc,idx,instrument):
    session = sessionmaker()
    session.configure(bind=engine)
    s = session()
    sql = select([Metrics]).where(and_(Metrics.instrument_id == instrument,
        Metrics.Metric == metrc,Metrics.equity_fundamental_id == idx)).select_from(Metrics)
    result=s.execute(sql)
    df_ = pd.DataFrame(result.fetchall()).drop_duplicates()
    df_.columns = result.keys()
    s.close()
    return df_

def get_metrics(instrument):
    session = sessionmaker()
    session.configure(bind=engine)
    s = session()
    sql = select([Metrics]).where(Metrics.instrument_id == instrument).select_from(Metrics)
    result=s.execute(sql)
    df_ = pd.DataFrame(result.fetchall()).drop_duplicates()
    df_.columns = result.keys()
    s.close()
    return df_

In [ ]:
df1=get_metric_ts("Open",2,379)
df2=get_metric_ts("Market Cap",1,379)
df3=get_metric_ts("Day's Range",2,379)
df4=get_metric_ts("Avg. Volume",2,379)
df5=get_metric_ts("1y Target Est",1,379)
df6=get_metric_ts("Enterprise Value 3",2,379)

df7=get_metric_ts("Trailing P/E",2,379)
df8=get_metric_ts("Forward P/E 1",2,379)
df9=get_metric_ts("No. of Analysts",8,379)
df10=get_metric_ts("Avg. Estimate",8,379)
df11=get_metric_ts("Low Estimate",8,379)
df12=get_metric_ts("High Estimate",8,379)
df13=get_metric_ts("Year Ago EPS",8,379)

In [ ]:
df4

In [ ]:
result=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13])
result=result[(result.Period!="Next Qtr. (Sep 2018)")&(result.Period!="Current Year (2018)")&(result.Period!="Next Year (2019)")]
result=result.sort_values(["refreshed_at","Metric"])

instrument=get_data(Instrument)[["id","name"]]
equity=get_data(Equity_fundamentals)

result=pd.merge(result,instrument,left_on="instrument_id",right_on="id")
result=pd.merge(result,equity,left_on="equity_fundamental_id",right_on="id")
result=result.sort_values(["refreshed_at","Metric"])
result=result[["name","Category","Table","Metric","Period","Value","refreshed_at"]]
result.columns=["Ticker","Category","Section","Metric","Period","Value","Date"]
#result.set_index("Date")
#result["Date2"]=pd.to_datetime(result['Date'])#, format='%Y-%m-%d %H:%M:%S')
result=result.sort_values(["Date"])

In [ ]:
#result['Date']

In [ ]:
#result["DATE2"]= ts.dt.date
#result["DATE2"]=pd.to_datetime(result['Date'], format="%Y-%m-%d")
result["DATE2"]=result['Date'].dt.date.astype(str)

In [ ]:
result.head()

In [ ]:
result[result.DATE2=="2018-05-20"]

In [ ]:
result.DATE2.drop_duplicates().shape

In [ ]:
#result.groupby(['DATE2']).count()
result.groupby(['Metric']).count()

In [ ]:
result.to_csv("SBUX.csv")

In [3]:
all_data=get_metrics(379)

In [5]:
all_data.head()
instrument=get_data(Instrument)[["id","name"]]
equity=get_data(Equity_fundamentals)

result=pd.merge(all_data,instrument,left_on="instrument_id",right_on="id")
result=pd.merge(result,equity,left_on="equity_fundamental_id",right_on="id")
result=result.sort_values(["refreshed_at","Metric"])
result=result[["name","Category","Table","Metric","Period","Value","refreshed_at"]]
result.columns=["Ticker","Category","Section","Metric","Period","Value","Date"]
#result.set_index("Date")
#result["Date2"]=pd.to_datetime(result['Date'])#, format='%Y-%m-%d %H:%M:%S')
result=result.sort_values(["Date"])
result.head()

,Ticker,Category,Section,Metric,Period,Value,Date
1666,SBUX,Statistics,Trading Information,% Held by Insiders 1,2018-05-20,2.88%,2018-05-20 08:04:45
0,SBUX,Summary,Summary,Market Cap,2018-05-20,78.881B,2018-05-20 08:04:45
328,SBUX,Statistics,Valuation Measures,Market Cap (intraday) 5,2018-05-20,78.88B,2018-05-20 08:04:45
691,SBUX,Statistics,Financial Highlights,Most Recent Quarter (mrq),2018-05-20,"Apr 1, 2018",2018-05-20 08:04:45
701,SBUX,Statistics,Financial Highlights,Net Income Avi to Common (ttm),2018-05-20,4.39B,2018-05-20 08:04:45


In [6]:
result.to_csv("SBUX.csv")

(15224, 7)

#  TEST DOWNLOAD OPTIONS INFO FROM YAHOO

In [ ]:
import sys
sys.path.append('/home/ubuntu/quantum/Infrastructure')
#from download_historical_equity_fundamental import *
from Schema import *
from mvo_utils import *
sys.path.append('home/ubuntu/quantum/Modeling')

In [ ]:
def get_summary(name):
    try:
        name1=name.replace('WIKI/' ,'')
        summaryf=get_data_sf(pd.read_html('https://finance.yahoo.com/quote/'+name1+'/options?p='+name1))[1]
        summaryf.columns=["Metric","Value"]
        summaryf["Category"]="Summary";summaryf["Table"]="Summary"
        summaryf["Period"]=str(datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d'))
        summaryf["refreshed_at"]=datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
        summaryf["instrument_id"]=get_id_instrument(name)
        summaryf["data_vendor_id"]=get_id_data_vendor("Yahoo_Finance")
        summaryf["equity_fundamental_id"]=summaryf[["Category","Table"]].apply(lambda x :add_up_equityf(x["Category"],x["Table"]), axis=1)
        summaryf[["instrument_id","equity_fundamental_id","data_vendor_id","refreshed_at","Period","Metric","Value"]].to_csv(Path_stats+"summary_"+name1+".csv")
        return True
    except Exception as err:
        print("Error: {0}".format(err))
        print("No se existe summary para "+name)
        return err
        pass
    
# https://finance.yahoo.com/quote/SBUX/options?p=SBUX
def get_data_sf(data_request):
    count=0
    while True:
        try:
            count=count+1
            summary = data_request ; break
            if count==limit:
                break;
        except Exception as err:
            print("Error: {0}, waiting to try again in {1}".format(err, sleep_time))
            sleep(sleep_time)
            if count==limit:
                break;
    return summary

In [ ]:
get_summary("SBUX")

In [ ]:
name="SBUX"

name1=name.replace('WIKI/' ,'')
summaryf=get_data_sf(pd.read_html('https://finance.yahoo.com/quote/'+name1+'/options?p='+name1))#[1]

In [ ]:
summaryf

In [ ]:
from bs4 import BeautifulSoup as BS
url ="https://finance.yahoo.com/quote/SBUX/options?p=SBUX"
usock = urllib2.urlopen(url)
data = usock.read()
usock.close()
soup = BS(data)
#print soup.find('font', {'class':'big'}).text

#data-reactid="5"

In [ ]:
data

In [ ]:
soup

In [ ]:
import urllib
from lxml import html

#url = "https://finance.yahoo.com/quote/SBUX/options?p=SBUX"
#page = html.fromstring(urllib.urlopen(url).read())


import urllib.request
with urllib.request.urlopen("https://finance.yahoo.com/quote/SBUX/options?p=SBUX") as url:
    s = url.read()
    

#for link in page.xpath("//a"):
#    print "Name", link.text, "URL", link.get("href")

'''output >>
    [('Aathma Liyanage', 'athma.html'),
     ('Abewardhana Balasuriya', 'abewardhana.html'),
     ('Aelian Thilakeratne', 'aelian_thi.html'),
     ('Ahamed Mohideen', 'ahamed.html'),
    ]'''

In [ ]:
s